In [1]:
# ============================================================================
# LIEBHERR HACKATHON 2025 - HYBRID-MODELL (OPTIMIERT)
# ============================================================================
# Dieses Notebook enthält nur das beste Modell (Hybrid/Modell 4):
# - Adaptive Kombination von SOLL+Verzögerung und Fortschritt-basiert
# - Berücksichtigt individuellen Auftragsstatus
# - Beste Performance: ~34 Tage MAE (geschätzt)
# ============================================================================

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("LIEBHERR HACKATHON 2025 - HYBRID-MODELL")
print("="*80)
print("\n🎯 Ziel: Realistische Fertigstellungstermine für offene Aufträge")
print("📊 Methode: Adaptive Kombination von SOLL+Verzögerung und Fortschritt\n")


# ============================================================================
# TEIL 1: DATEN LADEN UND VORBEREITEN
# ============================================================================

print("="*80)
print("TEIL 1: DATEN LADEN")
print("="*80)

# Daten laden
df_history = pd.read_csv('df_history_clean.csv')
df_eval_public = pd.read_csv('df_eval_public_2025-11-03.csv')
df_eval_private = pd.read_csv('df_eval_private_2025-11-03.csv')
df_ids = pd.read_csv('df_IDs_for_eval_2025-11-03.csv')

print(f"✓ df_history geladen: {len(df_history):,} Zeilen")
print(f"✓ df_eval_public geladen: {len(df_eval_public):,} Zeilen")
print(f"✓ df_eval_private geladen: {len(df_eval_private):,} Zeilen")
print(f"✓ df_ids geladen: {len(df_ids):,} IDs")

# Kombiniere eval Dateien
df_eval = pd.concat([df_eval_public, df_eval_private], ignore_index=True)
print(f"✓ df_eval kombiniert: {len(df_eval):,} Zeilen")

# Datumsspalten konvertieren
date_cols_history = ['Auftragseingang', 'Auftragsende_SOLL', 'AFO_Start_SOLL',
                     'AFO_Ende_SOLL', 'AFO_Start_IST', 'AFO_Ende_IST', 'Auftragsende_IST']
date_cols_eval = ['Auftragseingang', 'Auftragsende_SOLL', 'AFO_Start_SOLL',
                  'AFO_Ende_SOLL', 'AFO_Start_IST']

for col in date_cols_history:
    if col in df_history.columns:
        df_history[col] = pd.to_datetime(df_history[col], errors='coerce')

for col in date_cols_eval:
    if col in df_eval.columns:
        df_eval[col] = pd.to_datetime(df_eval[col], errors='coerce')

print("✓ Datumsspalten konvertiert")

# Stichtag definieren
STICHTAG = pd.Timestamp('2024-03-01 14:30:00')
print(f"\n✓ Stichtag: {STICHTAG}")


# ============================================================================
# TEIL 2: HILFSFUNKTIONEN
# ============================================================================

print("\n" + "="*80)
print("TEIL 2: HILFSFUNKTIONEN")
print("="*80)

def add_business_hours(start_dt, hours):
    """
    Addiert Arbeitsstunden zu einem Startdatum.
    Arbeitszeit: Mo-Fr, 07:00-15:00 (8 Stunden/Tag)
    Feiertage werden gearbeitet.
    """
    if pd.isna(start_dt) or hours <= 0:
        return start_dt

    # Anzahl volle Arbeitstage
    full_days = int(hours // 8)
    remaining_hours = hours % 8

    # Normalisiere auf 07:00 des aktuellen Tages
    current = pd.Timestamp(start_dt.date()) + pd.Timedelta(hours=7)

    # Wenn Startzeit nach 07:00 ist, berücksichtige das
    if start_dt.hour >= 7 and start_dt.hour < 15:
        current = start_dt
    elif start_dt.hour >= 15:
        # Nach Feierabend -> nächster Tag 07:00
        current = pd.Timestamp(start_dt.date()) + pd.Timedelta(days=1, hours=7)
        # Wenn das ein Wochenende ist, springe zu Montag
        while current.weekday() >= 5:
            current += pd.Timedelta(days=1)

    # Addiere volle Arbeitstage
    days_added = 0
    while days_added < full_days:
        current += pd.Timedelta(days=1)
        if current.weekday() < 5:  # Mo-Fr
            days_added += 1

    # Addiere verbleibende Stunden
    if remaining_hours > 0:
        # Prüfe ob wir noch am selben Tag bleiben
        end_time = current + pd.Timedelta(hours=remaining_hours)
        if end_time.hour > 15 or (end_time.hour == 15 and end_time.minute > 0):
            # Überschreitet Feierabend -> nächster Arbeitstag
            overflow = (end_time - current.replace(hour=15, minute=0, second=0)).total_seconds() / 3600
            current = current.replace(hour=15, minute=0, second=0)
            # Nächster Tag
            current += pd.Timedelta(days=1, hours=7)
            while current.weekday() >= 5:
                current += pd.Timedelta(days=1)
            current += pd.Timedelta(hours=overflow)
        else:
            current = end_time

    return current


def calculate_business_hours_between(start_dt, end_dt):
    """
    Berechnet Arbeitsstunden zwischen zwei Zeitpunkten.
    Mo-Fr, 07:00-15:00, Feiertage werden gearbeitet.
    """
    if pd.isna(start_dt) or pd.isna(end_dt):
        return 0

    if end_dt <= start_dt:
        return 0

    # Vereinfachte Berechnung: Zähle Werktage * 8h
    start_date = start_dt.date()
    end_date = end_dt.date()

    # Nutze numpy für effiziente Berechnung
    business_days = np.busday_count(start_date, end_date)

    return business_days * 8.0


print("✓ Hilfsfunktionen definiert:")
print("  - add_business_hours(): Addiert Arbeitsstunden")
print("  - calculate_business_hours_between(): Berechnet Arbeitsstunden zwischen Daten")


# ============================================================================
# TEIL 3: LEARNING - STATISTIKEN EXTRAHIEREN
# ============================================================================

print("\n" + "="*80)
print("TEIL 3: LEARNING - EXTRAHIERE STATISTIKEN")
print("="*80)

# 3.1: Standardarbeitsplan pro Bauteil
print("\n3.1 Standardarbeitsplan pro Bauteil...")

standard_afo_plan = df_history.groupby(['BauteilID', 'AuftragsID'])['Arbeitsschritt'].max().groupby('BauteilID').agg(['median', 'mean', 'max'])
standard_afo_plan.columns = ['Median_AFOs', 'Mean_AFOs', 'Max_AFOs']
print("\nAnzahl AFOs pro Bauteil:")
print(standard_afo_plan)

standard_afo_count = standard_afo_plan['Median_AFOs'].to_dict()

# 3.2: Durchschnittliche Dauer pro Bauteil
print("\n3.2 Berechne Gesamtdauer pro Bauteil...")

complete_orders = df_history[df_history['Auftragsende_IST'].notna()].copy()

order_durations = complete_orders.groupby('AuftragsID').agg({
    'BauteilID': 'first',
    'Bauteilbezeichnung': 'first',
    'AFO_Start_IST': 'min',
    'Auftragsende_IST': 'first'
}).reset_index()

order_durations['Duration_Hours'] = order_durations.apply(
    lambda row: calculate_business_hours_between(row['AFO_Start_IST'], row['Auftragsende_IST']),
    axis=1
)

# Entferne Outliers
order_durations = order_durations[
    (order_durations['Duration_Hours'] > 0) &
    (order_durations['Duration_Hours'] < 10000)
]

duration_stats = order_durations.groupby(['BauteilID', 'Bauteilbezeichnung'])['Duration_Hours'].agg([
    ('Count', 'count'),
    ('Median_Hours', 'median'),
    ('Mean_Hours', 'mean'),
    ('Std_Hours', 'std')
]).reset_index()

print("\nDauer-Statistiken:")
for _, row in duration_stats.iterrows():
    print(f"\n{row['Bauteilbezeichnung']} (ID={int(row['BauteilID'])}):")
    print(f"  Anzahl Aufträge: {int(row['Count']):>10,}")
    print(f"  Median Stunden:  {row['Median_Hours']:>10.1f} h ({row['Median_Hours']/8:.1f} Tage)")

median_duration_dict = duration_stats.set_index('BauteilID')['Median_Hours'].to_dict()

# 3.3: Durchschnittliche Verzögerung
print("\n3.3 Berechne Verzögerungen (SOLL vs IST)...")

orders_with_both = df_history[
    df_history['Auftragsende_IST'].notna() &
    df_history['Auftragsende_SOLL'].notna()
].copy()

delay_analysis = orders_with_both.groupby('AuftragsID').agg({
    'BauteilID': 'first',
    'Auftragsende_SOLL': 'first',
    'Auftragsende_IST': 'first'
}).reset_index()

delay_analysis['Delay_Days'] = (
    delay_analysis['Auftragsende_IST'] - delay_analysis['Auftragsende_SOLL']
).dt.total_seconds() / (24 * 3600)

delay_stats = delay_analysis.groupby('BauteilID')['Delay_Days'].agg([
    ('Count', 'count'),
    ('Median_Delay', 'median'),
    ('Mean_Delay', 'mean')
]).reset_index()

print("\nVerzögerungs-Statistiken (Tage):")
for _, row in delay_stats.iterrows():
    print(f"  BauteilID {int(row['BauteilID'])}: Median {row['Median_Delay']:>7.1f} Tage")

median_delay_dict = delay_stats.set_index('BauteilID')['Median_Delay'].to_dict()

# 3.4: AFO-Statistiken
print("\n3.4 Berechne AFO-Dauern...")

afo_durations = df_history[
    (df_history['AFO_Start_IST'].notna()) &
    (df_history['AFO_Ende_IST'].notna()) &
    (df_history['AFO_Dauer_IST_Stunde'].notna()) &
    (df_history['AFO_Dauer_IST_Stunde'] > 0) &
    (df_history['AFO_Dauer_IST_Stunde'] < 1000)
].copy()

afo_stats = afo_durations.groupby(['BauteilID', 'Arbeitsschritt'])['AFO_Dauer_IST_Stunde'].agg([
    ('Count', 'count'),
    ('Median_Hours', 'median')
]).reset_index()

print(f"✓ AFO-Statistiken für {len(afo_stats)} BauteilID-AFO Kombinationen")

afo_median_dict = afo_stats.set_index(['BauteilID', 'Arbeitsschritt'])['Median_Hours'].to_dict()

print("\n" + "="*80)
print("✅ LEARNING ABGESCHLOSSEN")
print("="*80)


# ============================================================================
# TEIL 4: FEATURE ENGINEERING
# ============================================================================

print("\n" + "="*80)
print("TEIL 4: FEATURE ENGINEERING FÜR HYBRID-MODELL")
print("="*80)

print("\nErstelle Features für 8.546 Aufträge...")

features_list = []

for idx, auftrag_id in enumerate(df_ids['AuftragsID']):
    if (idx + 1) % 1000 == 0:
        print(f"  Verarbeitet: {idx + 1}/{len(df_ids)}")

    # Hole eval-Info
    eval_row = df_eval[df_eval['AuftragsID'] == auftrag_id]
    if len(eval_row) == 0:
        continue
    eval_row = eval_row.iloc[0]

    # Hole Historie
    history = df_history[df_history['AuftragsID'] == auftrag_id].copy()

    # Basis-Features
    bauteil_id = eval_row['BauteilID']
    prioritaet = eval_row['Priorität']
    auftragsende_soll = eval_row['Auftragsende_SOLL']
    auftragseingang = eval_row['Auftragseingang']

    # Feature 1: Anzahl abgeschlossener AFOs
    completed_afos = len(history[history['AFO_Ende_IST'].notna()])

    # Feature 2: Standard AFO-Anzahl
    total_afos = standard_afo_count.get(bauteil_id, 10)

    # Feature 3: Fortschritt (0 bis 1)
    progress = completed_afos / total_afos if total_afos > 0 else 0

    # Feature 4: Letzter bekannter Zeitpunkt
    if len(history) > 0 and history['AFO_Ende_IST'].notna().any():
        last_afo_end = history['AFO_Ende_IST'].max()
    else:
        last_afo_end = STICHTAG

    # Feature 5: Bisherige Gesamtdauer
    if len(history) > 0 and history['AFO_Start_IST'].notna().any():
        first_start = history['AFO_Start_IST'].min()
        elapsed_hours = calculate_business_hours_between(first_start, last_afo_end)
    else:
        elapsed_hours = 0

    # Feature 6: Durchschnittliche AFO-Dauer bisher
    if completed_afos > 0:
        completed_history = history[history['AFO_Dauer_IST_Stunde'].notna()]
        if len(completed_history) > 0:
            avg_afo_duration = completed_history['AFO_Dauer_IST_Stunde'].mean()
        else:
            avg_afo_duration = elapsed_hours / completed_afos if completed_afos > 0 else 0
    else:
        avg_afo_duration = 0

    # Feature 7: Verbleibende AFOs
    remaining_afos = max(0, total_afos - completed_afos)

    # Feature 8: Erwartete Median-Dauer
    expected_median_hours = median_duration_dict.get(bauteil_id, 1000)

    # Feature 9: Historische Verzögerung
    expected_delay_days = median_delay_dict.get(bauteil_id, 0)

    # Feature 10: Nächste AFO Info
    next_afo = eval_row['Arbeitsschritt']
    next_afo_started = not pd.isna(eval_row['AFO_Start_IST'])

    # Feature 11: Tage seit Auftragseingang
    days_since_entry = (STICHTAG - auftragseingang).days if not pd.isna(auftragseingang) else 0

    # Feature 12: Tage bis SOLL-Ende
    days_to_soll = (auftragsende_soll - STICHTAG).days if not pd.isna(auftragsende_soll) else 100

    features = {
        'AuftragsID': auftrag_id,
        'BauteilID': bauteil_id,
        'Prioritaet': prioritaet,
        'Completed_AFOs': completed_afos,
        'Total_AFOs': total_afos,
        'Remaining_AFOs': remaining_afos,
        'Progress': progress,
        'Last_AFO_End': last_afo_end,
        'Elapsed_Hours': elapsed_hours,
        'Avg_AFO_Duration': avg_afo_duration,
        'Expected_Median_Hours': expected_median_hours,
        'Expected_Delay_Days': expected_delay_days,
        'Next_AFO': next_afo,
        'Next_AFO_Started': next_afo_started,
        'Days_Since_Entry': days_since_entry,
        'Days_To_SOLL': days_to_soll,
        'Auftragsende_SOLL': auftragsende_soll
    }

    features_list.append(features)

df_features = pd.DataFrame(features_list)
print(f"\n✓ Features erstellt für {len(df_features)} Aufträge")

print("\n📊 Feature-Statistiken:")
print(df_features[['Progress', 'Remaining_AFOs', 'Avg_AFO_Duration', 'Expected_Delay_Days']].describe())


# ============================================================================
# TEIL 5: HYBRID-MODELL (ADAPTIVE PROGNOSE)
# ============================================================================

print("\n" + "="*80)
print("TEIL 5: HYBRID-MODELL - ADAPTIVE PROGNOSE")
print("="*80)

print("\n🎯 Strategie:")
print("  - Fortschritt < 50%: 30% Performance + 70% SOLL+Verzögerung")
print("  - Fortschritt > 50%: 70% Performance + 30% SOLL+Verzögerung")
print("  → Nutzt jeweils die zuverlässigste Information!\n")


def predict_soll_plus_delay(row):
    """
    Modell-Komponente 1: SOLL + Historische Verzögerung
    """
    if not pd.isna(row['Auftragsende_SOLL']):
        return row['Auftragsende_SOLL'] + pd.Timedelta(days=row['Expected_Delay_Days'])
    else:
        return STICHTAG + pd.Timedelta(days=100)


def predict_progress_based(row):
    """
    Modell-Komponente 2: Fortschritt-basiert
    """
    start_point = row['Last_AFO_End']

    # Geschätzte verbleibende Zeit
    if row['Progress'] > 0 and row['Avg_AFO_Duration'] > 0:
        # Nutze bisherige Performance
        estimated_remaining = row['Remaining_AFOs'] * row['Avg_AFO_Duration']
    else:
        # Fallback: Nutze Median pro AFO
        avg_hours_per_afo = row['Expected_Median_Hours'] / row['Total_AFOs'] if row['Total_AFOs'] > 0 else 100
        estimated_remaining = row['Remaining_AFOs'] * avg_hours_per_afo

    # Sicherheitsfaktor: Addiere 10% Buffer
    estimated_remaining *= 1.1

    # Berechne End-Datum
    predicted_end = add_business_hours(start_point, estimated_remaining)

    return predicted_end


def predict_hybrid(row):
    """
    HYBRID-MODELL: Adaptive Kombination
    Gewichtung abhängig vom Fortschritt
    """
    # Berechne beide Komponenten
    pred_soll_delay = predict_soll_plus_delay(row)
    pred_progress = predict_progress_based(row)

    # Adaptive Gewichtung
    if row['Progress'] > 0.5:
        # Weit fortgeschritten → vertraue mehr auf Performance
        weight_progress = 0.7
    else:
        # Am Anfang → vertraue mehr auf SOLL-Planung
        weight_progress = 0.3

    # Berechne gewichteten Durchschnitt
    pred_soll_ts = pred_soll_delay.value if hasattr(pred_soll_delay, 'value') else pd.Timestamp(pred_soll_delay).value
    pred_prog_ts = pred_progress.value if hasattr(pred_progress, 'value') else pd.Timestamp(pred_progress).value

    hybrid_ts = weight_progress * pred_prog_ts + (1 - weight_progress) * pred_soll_ts

    return pd.Timestamp(hybrid_ts)


# Erstelle Prognosen
print("Erstelle Hybrid-Prognosen...")
df_features['Prediction_Hybrid'] = df_features.apply(predict_hybrid, axis=1)

print("✓ Prognosen erstellt für alle 8.546 Aufträge")

# Statistiken
print("\n📊 Prognose-Statistiken:")
avg_date = df_features['Prediction_Hybrid'].mean()
min_date = df_features['Prediction_Hybrid'].min()
max_date = df_features['Prediction_Hybrid'].max()

print(f"  Durchschnitt:  {avg_date.strftime('%Y-%m-%d')}")
print(f"  Früheste:      {min_date.strftime('%Y-%m-%d')}")
print(f"  Späteste:      {max_date.strftime('%Y-%m-%d')}")

# Verteilung nach Fortschritt
print("\n📊 Prognosen nach Fortschritt:")
for progress_range, label in [((0, 0.2), '0-20%'), ((0.2, 0.5), '20-50%'),
                               ((0.5, 0.8), '50-80%'), ((0.8, 1.0), '80-100%')]:
    mask = (df_features['Progress'] >= progress_range[0]) & (df_features['Progress'] < progress_range[1])
    count = mask.sum()
    if count > 0:
        avg = df_features[mask]['Prediction_Hybrid'].mean()
        print(f"  {label:8} ({count:>4} Aufträge): Ø {avg.strftime('%Y-%m-%d')}")


# ============================================================================
# TEIL 6: SUBMISSION ERSTELLEN
# ============================================================================

print("\n" + "="*80)
print("TEIL 6: SUBMISSION-DATEI ERSTELLEN")
print("="*80)

# Erstelle Submission DataFrame
df_submission = df_features[['AuftragsID', 'Prediction_Hybrid']].copy()
df_submission.columns = ['AuftragsID', 'Auftragsende_PREDICTED']

# ID-Spalte hinzufügen
df_submission.insert(0, 'ID', np.arange(1, len(df_submission) + 1))

# Formatiere Datum (nur YYYY-MM-DD, keine Zeit!)
df_submission['Auftragsende_PREDICTED'] = pd.to_datetime(
    df_submission['Auftragsende_PREDICTED']
).dt.strftime('%Y-%m-%d')

# Validierung
print("\n✓ Validierung:")
print(f"  1. Alle IDs vorhanden? {len(df_submission) == len(df_ids)}")
print(f"  2. Spalten korrekt? {list(df_submission.columns) == ['ID', 'AuftragsID', 'Auftragsende_PREDICTED']}")
print(f"  3. Keine NaN? {df_submission.isnull().sum().sum() == 0}")

# Datumsformat prüfen
date_pattern = r'^\d{4}-\d{2}-\d{2}$'
date_format_ok = df_submission['Auftragsende_PREDICTED'].str.match(date_pattern).all()
print(f"  4. Datumsformat YYYY-MM-DD? {date_format_ok}")

# Speichern
output_file = 'submission_hybrid.csv'
df_submission.to_csv(output_file, index=False)

print(f"\n✅ Submission gespeichert: {output_file}")
print(f"   Format: CSV")
print(f"   Separator: ,")
print(f"   Zeilen: {len(df_submission):,}")
print(f"   Spalten: ID, AuftragsID, Auftragsende_PREDICTED")

# Preview
print(f"\n📋 Erste 10 Zeilen:")
print(df_submission.head(10))

print(f"\n📋 Letzte 5 Zeilen:")
print(df_submission.tail(5))


# ============================================================================
# TEIL 7: ZUSAMMENFASSUNG & NÄCHSTE SCHRITTE
# ============================================================================

print("\n" + "="*80)
print("🎉 HYBRID-MODELL ABGESCHLOSSEN")
print("="*80)

print(f"""
📊 ZUSAMMENFASSUNG:

Daten verarbeitet:
├─ Historie:           {len(df_history):,} Zeilen
├─ Aufträge analyzed:  {len(order_durations):,}
└─ Prognosen:          {len(df_submission):,}

Modell-Strategie:
├─ Komponente 1: SOLL + historische Verzögerung
├─ Komponente 2: Fortschritt-basierte Schätzung
└─ Hybrid: Adaptive Gewichtung je nach Fortschritt

Features pro Auftrag:
├─ Fortschritt (abgeschlossene/verbleibende AFOs)
├─ Bisherige Performance (Ø AFO-Dauer)
├─ Historische Benchmarks (Median-Dauer, Verzögerung)
└─ Zeitliche Faktoren (Tage bis SOLL, seit Eingang)

Prognose-Verteilung:
├─ Durchschnitt:  {avg_date.strftime('%Y-%m-%d')}
├─ Bereich:       {min_date.strftime('%Y-%m-%d')} bis {max_date.strftime('%Y-%m-%d')}
└─ Individuell:   Jeder Auftrag basierend auf seinem Status

🎯 NÄCHSTE SCHRITTE:

1. Datei auf Kaggle hochladen:
   → submission_hybrid.csv

2. MAE im Public Leaderboard prüfen
   → Erwarteter MAE: ~30-40 Tage
   → Verbesserung gegenüber Baseline: ~60%

3. Bei Bedarf iterieren:
   → Gewichtungsfaktoren anpassen
   → Sicherheitspuffer optimieren
   → Weitere Features hinzufügen

4. Für finale Bewertung:
   → 2 beste Submissions markieren
   → Private Leaderboard abwarten

💡 TIPPS FÜR WEITERE VERBESSERUNGEN:

- Maschinenauslastungs-Features
- Wochentag-spezifische Adjustments
- Prioritäts-Gewichtung
- Machine Learning (XGBoost/Random Forest)
- Ensemble mit anderen Ansätzen

✅ BEREIT FÜR KAGGLE-SUBMISSION!
""")

print("="*80)
print("Viel Erfolg beim Hackathon! 🚀")
print("="*80)

LIEBHERR HACKATHON 2025 - HYBRID-MODELL

🎯 Ziel: Realistische Fertigstellungstermine für offene Aufträge
📊 Methode: Adaptive Kombination von SOLL+Verzögerung und Fortschritt

TEIL 1: DATEN LADEN
✓ df_history geladen: 1,360,869 Zeilen
✓ df_eval_public geladen: 4,273 Zeilen
✓ df_eval_private geladen: 4,273 Zeilen
✓ df_ids geladen: 8,546 IDs
✓ df_eval kombiniert: 8,546 Zeilen
✓ Datumsspalten konvertiert

✓ Stichtag: 2024-03-01 14:30:00

TEIL 2: HILFSFUNKTIONEN
✓ Hilfsfunktionen definiert:
  - add_business_hours(): Addiert Arbeitsstunden
  - calculate_business_hours_between(): Berechnet Arbeitsstunden zwischen Daten

TEIL 3: LEARNING - EXTRAHIERE STATISTIKEN

3.1 Standardarbeitsplan pro Bauteil...

Anzahl AFOs pro Bauteil:
           Median_AFOs  Mean_AFOs  Max_AFOs
BauteilID                                  
1                999.0      999.0       999
2                999.0      999.0       999
3                999.0      999.0       999

3.2 Berechne Gesamtdauer pro Bauteil...

Dauer-Sta